In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
#from imdb import IMDB
import sys

In [2]:
torch.__version__
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [3]:
sys.getdefaultencoding()

'utf-8'

In [4]:
is_cuda = False

if torch.cuda.is_available():
    is_cuda=True

In [5]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

In [6]:
train, test = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.2MB/s]


In [7]:
type(train)

torchtext.datasets.imdb.IMDB

In [8]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

train.fields {'text': <torchtext.data.field.Field object at 0x7f33923fd1d0>, 'label': <torchtext.data.field.Field object at 0x7f33923fd240>}
len(train) 25000
vars(train[0]) {'text': ['this', 'may', 'just', 'be', 'the', 'most', 'nostalgic', 'journey', 'back', 'in', 'time', '&', 'through', 'time', 'to', 'when', "one's", 'childhood', 'starts', 'a', 'journey', 'to', 'reminiscences', 'back', '&', 'forth', 'onwards', '&', 'upwards,forwards', '&', 'backwards,up', '&', 'down', '&', 'all', 'around.the', 'boy', 'jimmy,h.r.', 'puffinstuff,dr.blinky,cling', '&', 'clang,ludicrous', 'lion,&', 'even', 'the', 'evil', 'witchie', 'poo', 'too', 'through', '&', 'through.', 'the', 'latter', 'day', 'inspirations', 'of', 'lidsville,"the', 'brady', 'kids', 'saturday', 'morning', 'preview', 'special"', 'sigmund', '&', 'the', 'sea', 'monsters,and', 'land', 'of', 'the', 'lost', 'both', 'the', 'new', '&', 'old', 'are', 'what', 'this', 'very', 'show', 'bridged', 'the', 'gap', 'to', 'as', 'well', 'as', 'the', 'donn

In [9]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 399335/400000 [00:42<00:00, 9556.78it/s]

In [10]:
LABEL.vocab.freqs

Counter({'neg': 12500, 'pos': 12500})

In [11]:
d = vars(TEXT.vocab)

In [12]:
d.keys()

dict_keys(['freqs', 'itos', 'stoi', 'vectors'])

In [13]:
TEXT.vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7724, -0.1800,  0.2072,  ...,  0.6736,  0.2263, -0.2919],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [14]:
len(TEXT.vocab.stoi)

10002

In [32]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device="cuda")

train_iter.repeat = False
test_iter.repeat = False

In [33]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc = nn.Linear(hidden_size2,3)
        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc(embeds)
        return F.log_softmax(out,dim=-1)

In [34]:
model = EmbNet(len(TEXT.vocab.stoi),10)
model = model.cuda()

In [35]:
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [38]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,reduction="sum").item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [39]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.43 and training accuracy is 20044/25000     80.18
validation loss is  0.62 and validation accuracy is 17374/25000      69.5
training loss is   0.4 and training accuracy is 20404/25000     81.62
validation loss is  0.64 and validation accuracy is 17390/25000     69.56
training loss is  0.38 and training accuracy is 20771/25000     83.08
validation loss is  0.65 and validation accuracy is 17450/25000      69.8
training loss is  0.36 and training accuracy is 21068/25000     84.27
validation loss is  0.67 and validation accuracy is 17492/25000     69.97
training loss is  0.33 and training accuracy is 21403/25000     85.61
validation loss is  0.69 and validation accuracy is 17444/25000     69.78
training loss is  0.31 and training accuracy is 21668/25000     86.67
validation loss is  0.71 and validation accuracy is 17462/25000     69.85
training loss is  0.29 and training accuracy is 21992/25000     87.97
validation loss is  0.74 and validation accuracy is 17418/25000   

In [41]:
#Using Pretrained Glove Word Embeddings
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

train, test = datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train,test, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [42]:
class EmbNet2(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc1 = nn.Linear(hidden_size2,3)

        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc1(embeds)
        return F.log_softmax(out,dim=-1)

In [43]:
model = EmbNet2(len(TEXT.vocab.stoi),300,12000)
model = model.cuda()

In [44]:
model.embedding.weight.data = TEXT.vocab.vectors.cuda()

In [45]:
model.embedding.weight.requires_grad = False

In [46]:
optimizer = optim.Adam([ param for param in model.parameters() if param.requires_grad == True],lr=0.001)

In [48]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=64, device="cuda",shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

In [49]:
def fit2(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,reduction="sum").item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [50]:
train_losses2 , train_accuracy2 = [],[]
val_losses2 , val_accuracy2 = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit2(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit2(epoch,model,test_iter,phase='validation')
    train_losses2.append(epoch_loss)
    train_accuracy2.append(epoch_accuracy)
    val_losses2.append(val_epoch_loss)
    val_accuracy2.append(val_epoch_accuracy)


training loss is  0.66 and training accuracy is 15657/25000     62.63
validation loss is  0.64 and validation accuracy is 16350/25000      65.4
training loss is  0.56 and training accuracy is 17785/25000     71.14
validation loss is  0.65 and validation accuracy is 16505/25000     66.02
training loss is  0.53 and training accuracy is 18339/25000     73.36
validation loss is  0.68 and validation accuracy is 16464/25000     65.86
training loss is  0.51 and training accuracy is 18778/25000     75.11
validation loss is  0.69 and validation accuracy is 16434/25000     65.74
training loss is  0.49 and training accuracy is 19065/25000     76.26
validation loss is  0.73 and validation accuracy is 16214/25000     64.86
training loss is  0.47 and training accuracy is 19237/25000     76.95
validation loss is  0.74 and validation accuracy is 16246/25000     64.98
training loss is  0.47 and training accuracy is 19376/25000      77.5
validation loss is  0.77 and validation accuracy is 16240/25000   